In [ ]:
#Import all the librarys needed
import pandas as pd
import numpy as np
from scipy import stats
from numpy import mean
from numpy import std

In [ ]:
# Run previous ipynb files First then floowo grid in next step
%run extractData.ipynb

In [ ]:
# Check the data files in Grid View
MAIN_GRID[2][0]

In [ ]:
#load the data from repo to df
import pandas as pd
url ='https://raw.githubusercontent.com/zsaordenio/global-warming-usa/master/data/climateKnowledge/pr_1901_2016_USA.csv'
df1 = pd.read_csv(url, error_bad_lines=False)
df1.info() #checking the data types

In [ ]:
df1.columns

In [ ]:
#rename column name
df1 = df1.rename(index=str, columns={'Rainfall - (MM)': "Rnf_MM", " Year": "Year", })

In [ ]:
#Filter data set to 1990-2015 and the columns Year and Rainfall (MM)
#start = 1990
#end = 2015
#l = list(df1_filter[' Year']) #ordered list of years
#subl = l[l.index(start):l.index(end)+1] #list of years between the start and end
#df1_filter[df1_filter[' Year'].isin(subl)] #filter dataset for list of names
df1_filter = df1.iloc[0:,[0,1]]
df1_filter = (df1_filter.set_index(['Year']).loc[1990:2015]).reset_index()
df1_filter

In [ ]:
#load the data from repo to df
import pandas as pd
url1 ='https://github.com/zsaordenio/global-warming-usa/raw/master/data/climateKnowledge/tas_1901_2016_USA.csv'
df2 = pd.read_csv(url1, error_bad_lines=False)
df2.info() #checking the data types

In [ ]:
df2.columns
#df2_filter.info()

In [ ]:
#rename column name
df2 = df2.rename(index=str, columns={'Temperature - (Celsius)':"Tmp_Cls"," Year": "Year"})

In [ ]:
#Filter data set to 1990-2015 and the columns Year and Rainfall (MM)
df2_filter = df2.iloc[0:,[0,1]]
df2_filter = (df2_filter.set_index(['Year']).loc[1990:2015]).reset_index()
df2_filter

In [ ]:
#load the data from repo to df
import pandas as pd
url2 ='https://github.com/zsaordenio/global-warming-usa/raw/master/data/epa/ghg-concentrations.csv'
df3 = pd.read_csv(url2, error_bad_lines=False)
df3.info() #checking the data types

In [ ]:
df3.columns

In [ ]:
#rename column name
df3 = df3.rename(index=str, columns={"Year (negative values = BC)": "Year", "Mauna Loa, Hawaii": "MLHawai_CO2ppm"})
#Filter data set to columns Year and ml_hawai
df3_filter = df3.iloc[0:,[0,4]]
#convert everything to numerical values #https://stackoverflow.com/questions/47444999/check-if-column-contains-type-string-object
df3_filter.loc[:, df3_filter.dtypes.eq('object')] = df3_filter.loc[:, df3_filter.dtypes.eq('object')].apply(pd.to_numeric, errors='coerce')

#checking the data types
df3_filter.info()

#df3_filter.isnull().sum() #Checking if any null value exists.
#df3_filter.isna().sum() #Checking if any na value exists.

In [ ]:
#Filter data set to 1990-2015 and the columns Year and ml_hawai
df3_filter = (df3_filter.set_index(['Year']).loc[1990:2015]).reset_index()
df3_filter.info()

In [ ]:
df3_filter.isna().sum() #Checking if any null value exists.

In [ ]:
#Converting year to int64
df3_filter['Year'] = df3_filter['Year'].astype(np.int64) #casting float64 to int64

In [ ]:
df3_filter.info()

In [ ]:
df3_filter

In [ ]:
# Merge dataframes and fill the values that don't exist in the lines of merged dataframe simply fill with required strings as #https://stackoverflow.com/questions/44327999/python-pandas-merge-multiple-dataframes#:~:text=Just%20simply%20merge%20with%20DATE,to%20get%20all%20the%20data).&text=Now%2C%20basically%20load%20all%20the,using%20merge%20or%20reduce%20function.&text=Note%3A%20you%20can%20add%20as,frames%20inside%20the%20above%20list.
df_tobe_merged = [df1_filter, df2_filter, df3_filter]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Year'], how='outer'), df_tobe_merged).fillna('void')

df_merged

In [ ]:
df_merged.info()

In [ ]:
# summarize
from statistics import median 
print('mean=%.3f median =%.3f stdv=%.3f' % (mean(df1_filter["Rnf_MM"]), median(df1_filter["Rnf_MM"]), std(df1_filter["Rnf_MM"])))

In [ ]:
# histogram plot #Normality Check
import pandas as pd
import altair as alt
import numpy as np

alt.Chart(df1_filter).mark_bar().encode(
    alt.X("Rnf_MM:Q",
          bin= True), #alt.BinParams(maxbins=100)
    y='count(*):Q'
)

In [ ]:
#Normality Check # Hypothesis testing
from scipy import stats
k2, p = stats.normaltest(df1_filter["Rnf_MM"])
alpha = 1e-3
print("p = {:g}".format(p))

if p < alpha:  # null hypothesis: x comes from a normal distribution
    print("The null hypothesis can be rejected")
else:
    print("The null hypothesis cannot be rejected")
#The null hypothesis can be rejected

In [ ]:
#Scatter Plot # Distribution
import altair as alt
from vega_datasets import data


alt.Chart(df_merged).mark_circle().encode(
    alt.X(alt.repeat("column"), type='quantitative'),
    alt.Y(alt.repeat("row"), type='quantitative'),
    color='Year:O'
).properties(
    width=150,
    height=150
).repeat(
    column=['Rnf_MM','Tmp_Cls','MLHawai_CO2ppm'],
    row=['MLHawai_CO2ppm','Tmp_Cls','Rnf_MM']
).interactive()

In [ ]:
#Scatter Plot # Distribution
import altair as alt
from vega_datasets import data

alt.Chart(df_merged).mark_circle().encode(
    alt.X('Year:O', scale=alt.Scale(zero=False)),
    alt.Y('Tmp_Cls:Q', scale=alt.Scale(zero=False, padding=1)),
    color='MLHawai_CO2ppm:Q',
    size='Rnf_MM:Q'
)
